# Coronavirus Brasil vs Mundo - Gráficos Interativos
> Gráficos e análises com comparações entre Brasil e países específicos

- toc: false
- branch: master
- badges: false
- comments: false
- author: Vinicius Bastos Gomes & Fernando Wittmann
- categories: [coronavirus]


Fonte:  https://opendata.ecdc.europa.eu/covid19/casedistribution/csv


In [134]:
#hide
!pip install plotly --upgrade

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.6.0)


In [0]:
#hide_input
#Imports
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import numpy as np; np.random.seed(1)
import seaborn as sns
sns.set(style="ticks", rc={"lines.linewidth": 1.6})
sns.set_palette('bright')
import pandas as pd
from datetime import date,timedelta
import io

# Montar google drive e exportar imagens
MONTAR_GDRIVE = True


In [136]:
#hide
if MONTAR_GDRIVE:
    # Montar Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
#hide
if MONTAR_GDRIVE:
    # Abrir pasta com os arquivos
    %cd "/content/drive/My Drive/CoronaVirus Brasil"

/content/drive/My Drive/CoronaVirus Brasil


In [0]:
#hide
!rm dados.csv

In [139]:
#hide
!wget https://opendata.ecdc.europa.eu/covid19/casedistribution/csv -O dados.csv

--2020-04-12 17:05:48--  https://opendata.ecdc.europa.eu/covid19/casedistribution/csv
Resolving opendata.ecdc.europa.eu (opendata.ecdc.europa.eu)... 212.181.0.63
Connecting to opendata.ecdc.europa.eu (opendata.ecdc.europa.eu)|212.181.0.63|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://opendata.ecdc.europa.eu/covid19/casedistribution/csv/ [following]
--2020-04-12 17:05:49--  https://opendata.ecdc.europa.eu/covid19/casedistribution/csv/
Reusing existing connection to opendata.ecdc.europa.eu:443.
HTTP request sent, awaiting response... 200 OK
Length: 528800 (516K) [application/octet-stream]
Saving to: ‘dados.csv’

dados.csv           100%[===================>] 516.41K   472KB/s    in 1.1s    

2020-04-12 17:05:50 (472 KB/s) - ‘dados.csv’ saved [528800/528800]



In [0]:
#hide
#datetime operations
todei = date.today()
today = todei.strftime("%d-%m-%Y")
today_us = todei.strftime("%Y-%m-%d")
today_us
yesterdei = date.today() - timedelta(days=1)
yesterday = yesterdei.strftime('%Y-%m-%d')

In [0]:
#hide
data3 = pd.read_csv('dados.csv', encoding='latin-1',error_bad_lines=False,parse_dates=['dateRep'],dayfirst=True)


In [0]:
#hide
def preprocessing(data):
  data.sort_index(ascending=False,inplace=True)
  data['Cumulativo'] = data['cases'].cumsum()
  data['deathcum'] = data['deaths'].cumsum()
  data = data[data['Cumulativo']>=50]
  data = data.reset_index(drop=True)

  return data

In [0]:
#hide
south_korea = data3[data3['countriesAndTerritories']=='South_Korea']
czech_Republic = data3[data3['countriesAndTerritories']=='Czechia']
hongkong = data3[data3['countriesAndTerritories']=='Hong_Kong']
taiwan = data3[data3['countriesAndTerritories']=='Taiwan']
mongolia = data3[data3['countriesAndTerritories']=='Mongolia']
japan = data3[data3['countriesAndTerritories']=='Japan']

In [0]:
#hide
china = data3[data3['countriesAndTerritories']=='China']
germany = data3[data3['countriesAndTerritories']=='Germany']
italy = data3[data3['countriesAndTerritories']=='Italy']
brazil = data3[data3['countriesAndTerritories']=='Brazil']
argentina = data3[data3['countriesAndTerritories']=='Argentina']
france = data3[data3['countriesAndTerritories']=='France']
us = data3[data3['countriesAndTerritories']=='United_States_of_America']
uk = data3[data3['countriesAndTerritories']=='United_Kingdom']
spain = data3[data3['countriesAndTerritories']=='Spain']
mexico = data3[data3['countriesAndTerritories']=='Mexico']
india = data3[data3['countriesAndTerritories']=='India']

In [0]:
#hide
countries = [brazil,germany,italy,argentina,france,us,uk,spain,south_korea,czech_Republic,hongkong,taiwan,mongolia,japan]

In [0]:
#hide
paises_list = []
for i in countries:
  y = preprocessing(i)
  paises_list.append(y)

In [147]:
#hide
paises = pd.concat(paises_list)
paises['indice'] = paises.index
paises['relativa'] = 10000*(paises['Cumulativo']/paises['popData2018'])
paises = paises.sort_values('indice')
paises['countriesAndTerritories'].unique()

array(['Brazil', 'Argentina', 'France', 'United_States_of_America',
       'United_Kingdom', 'Germany', 'South_Korea', 'Czechia', 'Taiwan',
       'Spain', 'Italy', 'Japan'], dtype=object)

In [0]:
final_data = paises

In [0]:
#hide
#add pct
brazil_dupl = paises[paises['countriesAndTerritories']=='Brazil']
brazil_dupl['razao']= brazil_dupl['Cumulativo'].pct_change()
brazil_dupl['razao']=(brazil_dupl['razao']*100).round(2)
brazil_dupl['dias']= brazil_dupl.index


<center> <h2> Gráficos de contaminação </h2> </center>




Os gráficos a seguir apresentam curvas de contaminação de diversos países. Foram criados com o intuito de comparar estratégias de combate à disseminação do vírus. É importante notar que as curvas de contaminação estão amplamente relacionadas à quantidade de testes que os países tem realizado.
A seção contem duas aproximações e duas versões do gáfico completo: nas escalas aritmética e logarítmica. 

In [150]:

#hide_input
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Argentina'], name='Argentina',line=dict(color='darkgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Czechia'], name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Taiwan'], name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Casos totais confirmados - {} \n Escala Aritmética'.format(today),
                   xaxis_title='Dias desde o caso 50',
                   yaxis_title='Casos')

In [151]:
#hide_input
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Argentina'], name='Argentina',line=dict(color='darkgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Czechia'], name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Taiwan'], name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['Cumulativo'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Casos totais confirmados - {} \n Escala Logarítmica'.format(today),
                   xaxis_title='Dias desde o caso 50',
                   yaxis_title='Casos',yaxis_type="log")

<center> <h2> Contaminação Relativa </h2> </center>



Essa seção conta com gráficos em que o número de casos é dívidido pela população dos países para que se tenha uma ideia da proporção de infectados por país.

In [152]:
#hide_input
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Argentina'], name='Argentina',line=dict(color='darkgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Czechia'], name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Taiwan'], name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Contaminação relativa  {} \nCasos a cada 10 mil habitantes - E. Aritmética' .format(today),
                   xaxis_title='Dias desde o caso 50',
                   yaxis_title='Casos')

In [153]:
#hide_input
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Argentina'], name='Argentina',line=dict(color='darkgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Czechia'], name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Taiwan'], name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['relativa'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Contaminação relativa  {} \nCasos a cada 10 mil habitantes - E. Logarítimica' .format(today),
                   xaxis_title='Dias desde o caso 50',
                   yaxis_title='Casos',yaxis_type="log")

<center> <h2> Mortes </h2> </center>

In [0]:
#hide
def preprocessing_death(data):
  data.sort_index(ascending=False,inplace=True)
  data['Cumulativo'] = data['cases'].cumsum()
  data['deathcum'] = data['deaths'].cumsum()
  data = data[data['deathcum']>=5]
  data = data.reset_index(drop=True)
  data = data.drop(['cases'],axis=1)
  return data

In [0]:
#hide
paises_death_list = []
for i in countries:
  y = preprocessing_death(i)
  paises_death_list.append(y)

In [156]:
#hide
paises = pd.concat(paises_death_list)
paises['indice'] = paises.index
paises['relativa'] = 10000*(paises['Cumulativo']/paises['popData2018'])
paises['letalidade']= 100*paises['deathcum']/paises['Cumulativo']
paises = paises.sort_values('indice')
paises['countriesAndTerritories'].unique()

array(['Brazil', 'United_Kingdom', 'Japan', 'France', 'Spain', 'Taiwan',
       'Argentina', 'South_Korea', 'Germany', 'United_States_of_America',
       'Czechia', 'Italy'], dtype=object)

In [0]:
#hide
final_data = paises
final_data = final_data.sort_values(['indice','countriesAndTerritories'])

In [194]:
#hide_input
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Argentina'],mode='lines', name='Argentina',line=dict(color='darkgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Czechia'],mode='lines', name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Taiwan'],mode='lines', name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Óbitos-  {} E. Aritmética' .format(today),
                   xaxis_title='Dias desde o 5º óbito',
                   yaxis_title='Casos')

In [193]:
#hide_input
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Argentina'], mode='lines',name='Argentina',line=dict(color='darkgreen', width=2,dash='solid')))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Czechia'],mode='lines', name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Taiwan'],mode='lines', name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['deathcum'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Óbitos-  {} E. Logarítmica' .format(today),
                   xaxis_title='Dias desde o 5º óbito',
                   yaxis_title='Casos',yaxis_type="log")

<center> <h2> Letalidade </h2> </center>

O gráfco a seguir aponta o número de mortes dividido pelo número de casos confirmados. É importantíssimo notar a grande dependência desses números da quantidade de testes disponíveis. Quanto mais testes realizados, mais confiáveis os dados.

In [171]:
#hide_input

fig = px.bar(final_data[final_data['dateRep']==yesterday],x="letalidade", y='countriesAndTerritories', color='countriesAndTerritories',orientation='h',
             color_discrete_sequence=px.colors.qualitative.G10,title="Letalidade - {}".format(today))#.update_yaxes(categoryorder='total dscending')
fig.update_layout(yaxis_type = 'category',)
fig.update_layout(legend= {'title':'País','borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})




In [196]:
#hide_input
fig = go.Figure()
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Brazil'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Brazil'], name='Brasil',line=dict(color='black', width=5)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='France'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='France'], name='França',line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Spain'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Spain'], name='Espanha',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Italy'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Italy'], name='Itália',line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_Kingdom'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='United_Kingdom'], name='Reino Unido',line=dict(color='yellowgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Argentina'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Argentina'], mode='lines', name='Argentina',line=dict(color='darkgreen', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Germany'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Germany'], name='Alemanha',line=dict(color='darkslateblue', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='United_States_of_America'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='United_States_of_America'], name='EUA',line=dict(color='mediumpurple', width=2)))


fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Czechia'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Czechia'], mode='lines', name='República Tcheca',line=dict(color='magenta', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='South_Korea'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='South_Korea'], name='Coréia do Sul',line=dict(color='brown', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Taiwan'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Taiwan'], mode='lines', name='Taiwan',line=dict(color='grey', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Hong_Kong'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Hong_Kong'], name='Hong Kong',line=dict(color='lavenderblush', width=2)))
fig.add_trace(go.Scatter(x=final_data['indice'][final_data['countriesAndTerritories']=='Japan'], 
                         y=final_data['letalidade'][final_data['countriesAndTerritories']=='Japan'], name='Japão',line=dict(color='pink', width=2)))



fig.update_layout(title='COVID-19 - Curvas de letalidade' .format(today),
                   xaxis_title='Dias desde o 5º óbito',
                   yaxis_title='Casos')

In [184]:
#hide
pro_bra = data3[data3['countriesAndTerritories']=='Brazil']
pro_bra['dateRep'] = pd.to_datetime(pro_bra['dateRep'],infer_datetime_format=True)
pro_bra.sort_values('dateRep',inplace=True)

pro_bra['death_cum'] = pro_bra['deaths'].cumsum()
pro_bra['case_cum'] = pro_bra['cases'].cumsum()

pro_bra.drop(['day', 'month','year',"geoId",'countryterritoryCode',"popData2018",'countriesAndTerritories'], axis = 1,inplace=True)
pro_bra.tail(4)

,dateRep,cases,deaths,death_cum,case_cum
1352,2020-04-09,2210,133,800,15927
1351,2020-04-10,1930,141,941,17857
1350,2020-04-11,1781,115,1056,19638
1349,2020-04-12,1089,68,1124,20727


In [185]:
#hide
pro_bra['Alemanha'] = pro_bra['death_cum']/let_ger*100
pro_bra = pro_bra[pro_bra['dateRep']>='2020-03-17']
pro_bra = pro_bra[pro_bra['dateRep']<=today]

pro_bra.sort_values('dateRep',ascending=False)
pro_bra.tail(3)

,dateRep,cases,deaths,death_cum,case_cum,Alemanha
1351,2020-04-10,1930,141,941,17857,42387.387387
1350,2020-04-11,1781,115,1056,19638,47567.567568
1349,2020-04-12,1089,68,1124,20727,50630.630631


O gráfico abaixo representa uma estimativa simples da quantidade de casos atualmente no Brasil. Uma vez que a baixa quantidade de testes tem levado os números a uma subnotificação, o cálculo foi realizado com base no número de óbitos registrados por COVID-19 no Brasil e na taxa da Alemanha, país que tem testado massivamente sua população. Desse modo os números são os seguintes:


In [187]:
#hide_input
print("Número atual de casos registrados no Brasil hoje: "+ str(pro_bra.iloc[-1,-2])  )
print("Estimativa de casos no Brasil hoje caso a letalidade seja próxima à da Alemanha: "+ str(int(pro_bra.iloc[-1,-1]))  )


Número atual de casos registrados no Brasil hoje: 20727
Estimativa de casos no Brasil hoje caso a letalidade seja próxima à da Alemanha: 50630


In [188]:
#hide_input
import plotly.graph_objects as go
pro_bra = pro_bra[pro_bra['dateRep']<=today_us]
fig = go.Figure()
fig.add_trace(go.Scatter(x=pro_bra['dateRep'], y=pro_bra['case_cum'], name='Brasil registrado',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=pro_bra['dateRep'], y=pro_bra['Alemanha'], name='Projeção Alemanha',
                         line=dict(color='blue', width=2)))


fig.update_layout(title='Projeção de casos no Brasil baseados nas taxas de letalidade da Alemanha',
                   xaxis_title='Data',
                   yaxis_title='Projeção')